In [6]:
#!pip install torchvision

In [1]:
from aicspylibczi import CziFile
import czifile
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import cv2
import os
import imageio
import ffmpeg
import time
import pandas as pd
from cellpose import io, models
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import json
import glob
import itertools
from PIL import Image
import torch.nn.functional as F
cudnn.benchmark = True
plt.ion()   # interactive mode

In [63]:
fake_data = [
    {
        "box": [(1, 2, 3, 4), (1, 2, 3, 5)],
        "masks": [np.ones([100, 100]), np.ones([100, 100])],
        "patch": [np.ones([100, 100]), np.ones([100, 100])]
    },
    {
        "box": [(1, 2, 3, 4), (1, 2, 3, 5)],
        "masks": [np.ones([100, 100]), np.ones((100, 100))],
        "patch": [np.ones([100, 100]), np.ones((100, 100))]
    },
    {
        "box": [(1, 2, 3, 4), (1, 2, 3, 5)],
        "masks": [np.ones([100, 100]), np.ones((100, 100))],
        "patch": [np.ones([100, 100]), np.ones((100, 100))]
    },
    {
        "box": [(1, 2, 3, 4), (1, 2, 3, 5)],
        "masks": [np.ones([100, 100]), np.ones((100, 100))],
        "patch": [np.ones([100, 100]), np.ones((100, 100))]
    },
    {
        "box": [(1, 2, 3, 4), (1, 2, 3, 5)],
        "masks": [np.ones([100, 100]), np.ones((100, 100))],
        "patch": [np.ones([100, 100]), np.ones((100, 100))]
    },
    {
        "box": [(1, 2, 3, 4), (1, 2, 3, 5)],
        "masks": [np.ones([100, 100]), np.ones((100, 100))],
        "patch": [np.ones([100, 100]), np.ones((100, 100))]
    },
    {
        "box": [(1, 2, 3, 4), (1, 2, 3, 5)],
        "masks": [np.ones([100, 100]), np.ones((100, 100))],
        "patch": [np.ones([100, 100]), np.ones((100, 100))]
    },
    {
        "box": [(1, 2, 3, 4), (1, 2, 3, 5)],
        "masks": [np.ones([100, 100]), np.ones((100, 100))],
        "patch": [np.ones([100, 100]), np.ones((100, 100))]
    },
    {
        "box": [(1, 2, 3, 4), (1, 2, 3, 5)],
        "masks": [np.ones([100, 100]), np.ones((100, 100))],
        "patch": [np.ones([100, 100]), np.ones((100, 100))]
    },
    {
        "box": [(1, 2, 3, 4), (1, 2, 3, 5)],
        "masks": [np.ones([100, 100]), np.ones((100, 100))],
        "patch": [np.ones([100, 100]), np.ones((100, 100))]
    }
    
]

fake_data[0]["masks"][1].flatten().shape

(10000,)

In [64]:
#get datasets
#one item is one cell
#data = {"patches" : [], 'boxes' : [], "masks" : []} 

# Create a lookup table to go between label name and index

class_names = "patches", "boxes", "masks"
num_classes = len(class_names)

id2label = {}
label2id = {}
for idx, label in enumerate(class_names):
    id2label[str(idx)] = label
    label2id[label] = str(idx)

In [65]:
box_shape = [100,100]
np.zeros((box_shape[0], box_shape[1])).flatten().shape

(10000,)

In [139]:
#dataset -  to do find the buggest boxes and add padding to everything
box_shape = (100, 100) #TO DO: find the biggest box and set it to this
T = 10

class CellBoxMaskPatch(torch.utils.data.Dataset):
    #input will be a Directory name, function is TO DO
    def __init__(
        self,
        vid_dir, 
        T=T):
        
        self.T = T
        

        # list of dictionaries, each dictionary is one cell tracked throughout a video
        #each cell has 3 data types in a dictionary
            #mask, box and patches
        self.cell_dict = self.get_dicts(vid_dir)

        #our "end tokens"
        self.end_box = (-1,-1,-1,-1)
        self.end_masks = np.zeros([box_shape[0], box_shape[1]]).flatten()
        self.end_patches = np.zeros([box_shape[0], box_shape[1]]).flatten()
        
        # How many total cells in dataset
        self.num_cells = len(self.cell_dict)
        self.num_timesteps, self.list_timesteps_start, self.list_timesteps_end = self.get_num_timesteps()

    def __len__(self):
        return self.num_timesteps

    def get_dicts(self, vid_dir):
        #returns a of list of dictionaries for every cell

        return fake_data
    
    def get_num_timesteps(self):
        #count the number of items in each list of each cell
        count= 0
        list_counts_start = [0]
        list_counts_end = []
        for cell in self.cell_dict:
            current_count = len(cell['box'])
            count+=current_count
            list_counts_end.append(count)
            list_counts_start.append(count)
        return count, list_counts_start[:-1], list_counts_end#list counts returns the start of every cell

    def binary_search(self, box_end, box_starts, number):
        #performs a binary search on the cells, to find the right timestep and cell number for a given index
        #if this gets slow, can edit to create a dictionary in get_num_timesteps
        left, right = 0, len(box_starts) - 1
        while left <= right:
            mid = left + (right - left) // 2
            if box_starts[mid] <= number < box_end[mid]:
                # Found the box
                box_number = mid  
                position_in_box = number - box_starts[mid]
                # print (number,  " found in box ", box_number, ", " , position_in_box)
                return box_number, position_in_box
            elif number < box_starts[mid]:
                right = mid - 1
            else:
                left = mid + 1
    
        # Number not found in any box
        raise ValueError(f"Number {number} is not found in any cell and timestep combination. Length shoudl be at most", self.num_timesteps)
        


    def __getitem__(self, idx, type='image'):
        #should return a dictionary for the cell, and T dictionaries for the next T cells. 
        #If any cells don't exist (ie we look for index n, n+1, n+2, n+3..) and the list is of 
        #length n+2, everything that doesn't exist (n+2, n+3...) should just be self.end, our "end token"
    
        #turn start and cell
        # print(self.list_timesteps_start, self.list_timesteps_end)

        cell_idx, start = self.binary_search(self.list_timesteps_end, self.list_timesteps_start, idx)

        cell = self.cell_dict[cell_idx]

        boxes = cell["box"]
        #flatten
        masks = cell["masks"]
        #flatten
        patches = cell["patch"]

        box = boxes[start]
        mask = masks[start]
        patch = patches[start]

        # Get the dictionaries for the next T cells
        next_T_frames_box = []
        next_T_frames_masks = []
        next_T_frames_patches = []
        for i in range(1, self.T + 1):
            # print("Getting next frame", i)
            if start + i < self.list_timesteps_end[cell_idx]-self.list_timesteps_start[cell_idx]:
                next_T_frames_box.append(boxes[start + i])
                next_T_frames_masks.append(masks[start+1].flatten())
                next_T_frames_patches.append(patches[start+i].flatten())
            else:
                # Append the default value `self.end` if the cell doesn't exist
                next_T_frames_box.append(self.end_box)
                next_T_frames_masks.append(self.end_masks)
                next_T_frames_patches.append(self.end_patches)
        
        # print("next T frames shape", np.array(next_T_frames_box).shape)
        #returns tuples, list of tuples
        return (box, mask, patch), (next_T_frames_box, next_T_frames_masks, next_T_frames_patches)

In [140]:
from torch.utils.data import random_split

cell_folder = "folder_with_cells_or_dicts_not_sure_implementation_rn"

dataset = CellBoxMaskPatch(cell_folder, 10) #looking only one timestep ahead

train, eval, test = random_split(dataset, [0.4, 0.2, 0.4])

input_datasets = {}
input_datasets["train"] = train
input_datasets["eval"] = eval
input_datasets["test"] = test

In [141]:
# timestep = [[0] for t in range(T)]
# timestep

# batch = ((np.array([1 , 2, 3]), 1, 2) , 3), ((np.array([1 , 2, 3]), 1, 2) , 3)
# current_boxes = torch.stack([torch.tensor(b[0][0], dtype=torch.long) for b in batch], dim=0)
# current_boxes
        #current     #next   #boxes                     #masks
batch  = [[1, 2, 3], [[[1,2, 3, 4], [1, 2, 3,4]], [[1, 2, 3], [1, 2, 3]], [[1, 2, 3], [1,2, 3]]]], [[1, 2, 3], [[[1,2, 3, 4], [1, 2, 3,4]], [[1, 2, 3], [1, 2, 3]], [[1, 2, 3], [1,2, 3]]]]



next_boxes = torch.stack([torch.tensor(b[1][0], dtype=torch.long) for b in batch], dim=0)


In [136]:
def collate_fn(batch, mode_box, mode_mask, mode_patch, T):
    #this should stick the batches together in way that makes sense in a torch stack manner
    #we want to return dim [B, 1, d], [B, T, d] where d is a combination of 4 (for boxes) and the flattened image shape
    #taking only taking one cell from sequence before and T is sequence after
    #if mode_box is true, we want to return [B, 1, 4] and [B, T, 4]
    #if mode_mask is true, we want to return [B, 1, (10,10).flatten] and [B, T, (10,10).flatten]
    #input (cur_box, cur_mask, cur_patch), (T_box, T_mask, T_patch)
  
    current_boxes = torch.stack([torch.tensor(b[0][0], dtype=torch.long) for b in batch], dim=0)
    current_masks = torch.stack([torch.tensor(b[0][1], dtype=torch.long) for b in batch], dim=0)
    current_patches = torch.stack([torch.tensor(b[0][2], dtype=torch.long) for b in batch], dim=0)
    selected_tensors = []


    next_boxes = torch.stack([torch.tensor(b[1][0], dtype=torch.int) for b in batch], dim=0)
    next_masks = torch.stack([torch.tensor(b[1][1], dtype=torch.float64) for b in batch], dim=0)
    next_patches = torch.stack([torch.tensor(b[1][2], dtype=torch.float64) for b in batch], dim=0)
    selected_next_tensors = []
    print("selected")

    if mode_box:
        selected_tensors.append(current_boxes)
        selected_next_tensors.append(next_boxes)
    if mode_mask:
        selected_tensors.append(current_masks)
        selected_next_tensors.append(next_masks)
    if mode_patch:
        selected_tensors.append(current_patches)
        selected_next_tensors.append(next_patches)


    # Concatenate selected tensors along the last dimension
    combined_tensor = torch.cat(selected_tensors, dim=-1)
    combined_next_tensors = torch.cat(selected_next_tensors, dim=-1)

    # Reshape to add the singleton dimension
    combined_tensor = combined_tensor.unsqueeze(1)

    # Verify the shape
    print("shape of input", combined_tensor.shape, "shape of labels of size", T, " ", combined_next_tensors.shape)


    return combined_tensor, combined_next_tensors
    
    

In [137]:
mode_box = True
mode_mask = False
mode_patch = False

dataloaders = {}
dataloaders['train'] = torch.utils.data.DataLoader(
    input_datasets['train'],
    batch_size=4,
    shuffle=True,
    num_workers=0,
    collate_fn=lambda batch: collate_fn(batch, mode_box, mode_mask, mode_patch, T)
)

dataloaders['test'] = torch.utils.data.DataLoader(
    input_datasets['test'],
    batch_size=4,
    shuffle=True,
    num_workers=0,
    collate_fn=lambda batch: collate_fn(batch, mode_box, mode_mask, mode_patch, T)
)

dataloaders['eval'] = torch.utils.data.DataLoader(
    input_datasets['eval'],
    batch_size=4,
    shuffle=True,
    num_workers=0,
    collate_fn=lambda batch: collate_fn(batch, mode_box, mode_mask, mode_patch, T)
)

In [138]:
for batch in dataloaders['train']:
    # input, output = batch
    # Process the batches as needed
    # print("Input Shape:", input.shape)
    # print("Output Shape:", output.shape)
    print("Input:", batch[0], "next ones", batch[1])
    
    break  

11
11
11
11
selected
shape of input torch.Size([4, 1, 4]) shape of labels of size 10   torch.Size([4, 10, 4])
Input: tensor([[[1, 2, 3, 5]],

        [[1, 2, 3, 4]],

        [[1, 2, 3, 4]],

        [[1, 2, 3, 4]]]) next ones tensor([[[-1, -1, -1, -1],
         [-1, -1, -1, -1],
         [-1, -1, -1, -1],
         [-1, -1, -1, -1],
         [-1, -1, -1, -1],
         [-1, -1, -1, -1],
         [-1, -1, -1, -1],
         [-1, -1, -1, -1],
         [-1, -1, -1, -1],
         [-1, -1, -1, -1]],

        [[ 1,  2,  3,  5],
         [-1, -1, -1, -1],
         [-1, -1, -1, -1],
         [-1, -1, -1, -1],
         [-1, -1, -1, -1],
         [-1, -1, -1, -1],
         [-1, -1, -1, -1],
         [-1, -1, -1, -1],
         [-1, -1, -1, -1],
         [-1, -1, -1, -1]],

        [[ 1,  2,  3,  5],
         [-1, -1, -1, -1],
         [-1, -1, -1, -1],
         [-1, -1, -1, -1],
         [-1, -1, -1, -1],
         [-1, -1, -1, -1],
         [-1, -1, -1, -1],
         [-1, -1, -1, -1],
         [-1,

In [221]:
#parameters for mask and patch
input_dim =  box_shape #TO DO: find the biggest box and set it to this
learning_rate = 0.0001
epochs = 25
T = 2 #len predictions
hidden_dims = 100
output_dim = input_dim


def sigmoid(x):
    return 1/ 1+ np.exp(-x)

In [ ]:
#create an LSTM RNN

#to create a pytorch LSTM 
#The first axis is the sequence itself
# the second indexes instances in the mini-batch
# third indexes elements of the input. 

# we would input T boxes

class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        #input_size, hidden_size, num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=2) #stacking 2 LSTMs

        # The linear layer that maps from hidden state space to tag space
        self.hidden2out = nn.Linear(hidden_dim, output_dim)

    def forward(self, input):
        lstm_out, h_n = self.lstm(input, batch_size, -1) #output and final hidden state
        output = self.hidden2out(lstm_out.view(len(input), -1))
        #output = F.log_softmax(output, dim=1)
        return output

In [272]:
import torch
import torch.nn as nn

class MyLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, batch_first=True):
        super(MyLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=batch_first)
        self.hidden2out = nn.Linear(hidden_size, input_size)


    def forward(self, x, T):
        # x has shape [B, 1, d]
        # Repeat the input tensor along the time dimension
        x = x.repeat(1, T, 1)
        # Pass through LSTM
        output, _ = self.lstm(x)
        output = self.hidden2out(output)
        return output

# Example input data
batch_size = 3
seq_length = 4
input_dim = 2
hidden_size = 5
num_layers = 1
input_data = torch.randn(batch_size, 1, input_dim)

# Create LSTM model
lstm_model = MyLSTM(input_dim, hidden_size, num_layers)

# Forward pass with desired sequence length T
T = seq_length
output = lstm_model(input_data, T)

print("Input shape:", input_data.shape)
print("Output shape:", output.shape)


Input shape: torch.Size([3, 1, 2])
Output shape: torch.Size([3, 4, 2])


In [ ]:
data_list = []
for _ in range(50):
    # Create a dictionary for each item
    item_dict = {
        "box": [(1, 2, 3, 4), (1, 2, 3, 5)],  # Example box values
        "masks": np.ones((100, 100)),  # Example masks array
        "patch": np.ones((100, 100))   # Example patch array
    }
    # Append the dictionary to the list
    data_list.append(item_dict)

# Print the first item to verify the structure
print(data_list[0])

{'box': [(1, 2, 3, 4), (1, 2, 3, 5)], 'masks': array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]]), 'patch': array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])}


In [ ]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

input_dim = 0
if mode_box == True:
    input_dim +=  4 #(T, 4) for the bounding boxes
if mode_mask == True:
    input_dim += box_shape.flatten #flattened version of [H, W]
if mode_patch == True:
    input_dim += box_shape.flatten #flattened version of [H, W]

learning_rate = 0.0001
epochs = 25
T = 2
hidden_dims = 100
output_dim = input_dim
batch_size = 1
model = LSTM(input_dim, hidden_dims, output_dim)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
# Do your training loop + eval, etc...
def train_model(model, criterion, optimizer, scheduler, num_epochs=epochs):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'dev']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]: #should be a [B, T, S] batch, time, sequence shape
                inputs = inputs.to(device)

                 # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs, labels) #B, T, [size of input]
                    _, preds = torch.max(outputs, dim=1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'dev' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())


    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model